In [1]:
import xml.etree.ElementTree as ET
import time
import pandas as pd

In [2]:
parking = ET.parse('Ch_950_Sch_34_MunicipalParkingFacilities.xml')
root = parking.getroot()
head = []

In [3]:
def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None

In [4]:
dfcols = ['Car_Park_No', 'Name', 'Address']
df_xml = pd.DataFrame(columns=dfcols)
for node in root:
    Car_Park_No = node.find('Car_Park_No')
    Name = node.find('Name')
    Address = node.find('Address_Location')
    
    df_xml = df_xml.append(pd.Series([getvalueofnode(Car_Park_No), getvalueofnode(Name),
                       getvalueofnode(Address)], index=dfcols),
            ignore_index=True)

In [26]:
from arcgis.gis import *
from arcgis.geocoding import geocode
dev_gis = GIS()

In [59]:
for index,row in df_xml.iterrows():
    address = row["Address"]+", Toronto"
    geocode_result = geocode(address)[0]
    geocode_result['location']
    lon = geocode_result['location']['x']
    lat = geocode_result['location']['y']
    df_xml.at[index,'lat'] = lat
    df_xml.at[index,'lon'] = lon

In [60]:
df_xml

,Car_Park_No,Name,Address,lat,lon
0,1,Hayden Street Garage,20 Charles Street East,43.668907,-79.385010
1,3,Isabella,13 Isabella Street,43.667701,-79.384744
2,5,Wellesley East,15 Wellesley Street East,43.665075,-79.383677
3,11,Rosehill Garage,21 Pleasant Boulevard,43.687292,-79.392835
4,12,Alvin Ave,30 Alvin Avenue,43.689545,-79.393156
5,13,DeLisle,10 DeLisle Avenue,43.689039,-79.395098
6,15,Yorkville-Cumberland Garage,37 Yorkville Avenue,43.671378,-79.389928
7,17,Pape North,716 Pape Avenue,43.679576,-79.345146
8,18,Keele St,351 Keele Street,43.664711,-79.464228
9,19,Pacific,385 Pacific Avenue,43.664912,-79.468328


In [65]:
import pygeohash as gh
df_xml['geohash']=df_xml.apply(lambda x: gh.encode(x.lat, x.lon, precision=7), axis=1)

In [67]:
df_xml.to_csv("Chapter_950_MunicipalParkingFacilities.csv")